In [1]:
# 安装必要的库 (如果已安装可跳过)
#!pip install openai pandas tqdm

In [2]:
import os
import json
import pandas as pd
from openai import OpenAI
from tqdm import tqdm # 这是一个进度条工具，很方便查看进度

# 设置你的 API Key
# 建议直接把 Key 粘贴在这里，或者从环境变量读取
# 如果使用的是 DeepSeek 或其他兼容 OpenAI 格式的模型，也可以在这里改 base_url
api_key = "sk-..."  # 🔴 请在这里替换你的 OpenAI API Key
client = OpenAI(api_key=api_key)

print("库导入成功，API 客户端已就绪！")

库导入成功，API 客户端已就绪！


In [3]:
# 读取数据
df_cambridge = pd.read_csv('cleaned_cambridge.csv')

# 🔴 关键设置：
# 测试模式：只取前 3 条，验证代码能否跑通，防止浪费钱或报错
# 这里的 .head(3) 意味着只处理前 3 行。
# 如果测试成功，你需要把 .head(3) 删掉，改成 df_cambridge.copy() 来跑全量
test_data = df_cambridge.head(3).copy() 

print(f"Cambridge 数据集总行数: {len(df_cambridge)}")
print(f"当前准备处理的行数: {len(test_data)}")
print("预览数据:")
display(test_data.head())

Cambridge 数据集总行数: 331
当前准备处理的行数: 3
预览数据:


,text,cefr_level
0,New books this month\n\nThe Long Night\nThis i...,A2
1,BURGLARS LOVE THE AFTERNOON\n\nMost house burg...,A2
2,Madame Tussaud's\n\nOne very famous place for ...,A2


生成 A1-C1 多难度平行语料 (Cambridge版)
Deepseek API

In [12]:
import os
import json
import pandas as pd
from openai import OpenAI
from tqdm import tqdm

# Using deepseek
API_KEY = "sk-cd3c66b5a2db4f0386ab67b52e6e3196"  # DeepSeek Key
client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")

# define all levels
ALL_LEVELS = ["A1", "A2", "B1", "B2", "C1", "C2"]

# core function
def generate_cefr_versions(original_text, original_level):
    """
    input: original text
    generate all other levels of text
    """
    
    # 1. Target Levels: if the goal level=original level, than skip the transform
    if original_level in ALL_LEVELS:
        target_levels = [lvl for lvl in ALL_LEVELS if lvl != original_level]
    else:
        target_levels = ALL_LEVELS

    target_str = ", ".join(target_levels)

    # 2. Prompt
    system_prompt = f"""
    You are an expert in CEFR-graded English text simplification.
    Your task is to rewrite the provided text into these specific levels: {target_str}.
    
    Strict Constraints:
    1. Keep ALL facts, entities, and meaning IDENTICAL to the original.
    2. Output PURE JSON format with keys: {json.dumps(target_levels)}.
    3. Do NOT generate the level "{original_level}" because the user already has it.
    4. For lower levels, use simple syntax/vocab. For higher levels, use complex syntax/vocab.
    """

    user_prompt = f"Original Text ({original_level}):\n{original_text}\n\nRewrite into JSON format for: {target_str}."

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={ "type": "json_object" },
            temperature=0.3
        )
        content = response.choices[0].message.content
        result_dict = json.loads(content)
        
        # 3. put the original content back to dictionary
        if original_level in ALL_LEVELS:
            result_dict[original_level] = original_text
            
        return result_dict

    except Exception as e:
        print(f"Error: {e}")
        return None

# main procedure
df_cambridge = pd.read_csv('cleaned_cambridge.csv')

#  .copy()
test_data = df_cambridge.copy() 
# test_data = df_cambridge.copy() 

print(f"🚀 start mission: {len(test_data)} lines")

results = []

for index, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
    original_text = row['text']
    original_level = row['cefr_level']
    
    # original_level
    versions = generate_cefr_versions(original_text, original_level)
    
    if versions:
        result_entry = {
            "original_id": index,
            "original_text": original_text,
            "original_cefr": original_level,
            "text_A1": versions.get("A1"),
            "text_A2": versions.get("A2"),
            "text_B1": versions.get("B1"),
            "text_B2": versions.get("B2"),
            "text_C1": versions.get("C1"),
            "text_C2": versions.get("C2")
        }
        results.append(result_entry)

df_processed = pd.DataFrame(results)

print("\n✅ mission accomplished！")
display(df_processed.head())

df_processed.to_csv("cambridge_deepseek_augmented_test.csv", index=False)

🚀 start mission: 331 lines


 62%|██████████████████████▏             | 204/331 [2:44:03<3:49:08, 108.26s/it]

Error: Unterminated string starting at: line 107 column 40 (char 17027)


 63%|███████████████████████▎             | 208/331 [2:50:14<3:24:55, 99.96s/it]

Error: Unterminated string starting at: line 6 column 9 (char 12731)


...

 92%|███████████████████████████████████▊   | 304/331 [5:16:15<39:28, 87.73s/it]

Error: Unterminated string starting at: line 6 column 9 (char 16503)


100%|███████████████████████████████████████| 331/331 [5:42:20<00:00, 62.06s/it]


✅ mission accomplished！


,original_id,original_text,original_cefr,text_A1,text_A2,text_B1,text_B2,text_C1,text_C2
0,0,New books this month\n\nThe Long Night\nThis i...,A2,New books this month. The Long Night: This is ...,New books this month\n\nThe Long Night\nThis i...,New books this month. The Long Night: This is ...,New books this month. The Long Night: This is ...,New books this month. The Long Night: Marking ...,New books this month. The Long Night: This rep...
1,1,BURGLARS LOVE THE AFTERNOON\n\nMost house burg...,A2,Burglars like the afternoon. Police say most h...,BURGLARS LOVE THE AFTERNOON\n\nMost house burg...,Burglars prefer the afternoon. According to th...,Burglars favor the afternoon hours. Police rep...,Burglars exhibit a predilection for the aftern...,Burglars demonstrate a pronounced affinity for...
2,2,Madame Tussaud's\n\nOne very famous place for ...,A2,Madame Tussaud's is a famous place in London. ...,Madame Tussaud's\n\nOne very famous place for ...,Madame Tussaud's is a well-known tourist attra...,Madame Tussaud's museum is a prominent tourist...,Madame Tussaud's museum stands as an iconic at...,Madame Tussaud's museum epitomizes a quintesse...
3,3,Memo \n\nTo: All staff \nSubject: Holidays\nFr...,A2,Memo to all staff. Subject: Holidays. From: D ...,Memo \n\nTo: All staff \nSubject: Holidays\nFr...,Memo to all staff. Subject: Holidays. From: D ...,Memo to all staff. Subject: Holidays. From: D ...,Memo to all staff. Subject: Holidays. From: D ...,Memo to all staff. Subject: Holidays. From: D ...
4,4,CANADA GEESE\n\nCanada Geese are large blue an...,A2,Canada Geese are big birds. They are blue and ...,CANADA GEESE\n\nCanada Geese are large blue an...,Canada Geese are large birds with blue and whi...,Canada Geese are sizable birds characterized b...,"Canada Geese, distinguished by their substanti...","Canada Geese, notable for their considerable s..."


# clear those above original level

In [1]:
import pandas as pd

input_file = "cambridge_deepseek_augmented_test.csv"
df = pd.read_csv(input_file)


cefr_levels = ["A1", "A2", "B1", "B2", "C1", "C2"]
level_map = {lvl: i for i, lvl in enumerate(cefr_levels)}

def filter_simplification(row):
    """
    Core logic: Only retain texts with a difficulty level lower than original_cefr.    
    """
    original_lvl = row['original_cefr']
    
    # not in level_map than skip
    if original_lvl not in level_map:
        return row
    
    # original level score
    orig_idx = level_map[original_lvl]
    
    # text_A1, text_A2...
    for lvl in cefr_levels:
        target_idx = level_map[lvl]
        col_name = f'text_{lvl}'
        
        # target >= original level, delte (None）
        if target_idx >= orig_idx:
            row[col_name] = None
            
    return row

print("cleanning now...")
df_clean = df.apply(filter_simplification, axis=1)

# delete all A1 easiest level: no use
cols_to_check = [f'text_{lvl}' for lvl in cefr_levels]
df_final = df_clean.dropna(subset=cols_to_check, how='all')

#save results
output_file = "cambridge_simplified levels_only.csv"
df_final.to_csv(output_file, index=False)

print(f"✅ mission accomplished！")
print(f"original row: {len(df)}")
print(f"new row: {len(df_final)}")
print(f"save as: {output_file}")

# small check
print("\n preview level B2 (A1, A2, B1):")
b2_example = df_final[df_final['original_cefr'] == 'B2'].head(1)
if not b2_example.empty:
    cols = ['original_cefr', 'text_A1', 'text_A2', 'text_B1', 'text_B2', 'text_C1']
    print(b2_example[cols].to_string())

cleanning now...
✅ mission accomplished！
original row: 322
new row: 322
save as: cambridge_simplified levels_only.csv

 preview level B2 (A1, A2, B1):
    original_cefr                                                                                                                                                                                                                                                                                                                           text_A1                                                                                                                                                                                                                                                                                                                                                                                                                                                 text_A2                                                                      

In [4]:
!pip install torch transformers

# Evaluation: Clean those who do not satisfied with our Standard
Our device: Macbookpro m4pro 14CPU+20GPU core

In [ ]:
import pandas as pd
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# ========================================================
# 1. 终极稳定配置 (Python 3.10 + CPU + Verified Model)
# ========================================================

# 禁用并行，防止死锁
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 强制 CPU (在 Python 3.10 下非常稳定)
DEVICE = "cpu"
print(f"🛡️ Running on device: {DEVICE} (Stable Mode)")

# ✅ 这是一个真实存在、架构成熟的 CEFR 模型
CEFR_MODEL_NAME = "AbdulSami/bert-base-cased-cefr"

# ========================================================
# 2. 核心逻辑
# ========================================================

def load_model():
    print(f"Loading model: {CEFR_MODEL_NAME}...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(CEFR_MODEL_NAME)
        model = AutoModelForSequenceClassification.from_pretrained(CEFR_MODEL_NAME)
        model.to(DEVICE)
        model.eval()
        print("✅ 模型加载成功！(Python 3.10 环境验证通过)")
        return tokenizer, model
    except Exception as e:
        print(f"❌ 模型加载失败: {e}")
        return None, None

def predict_batch(texts, tok, model, batch_size=16):
    labels = []
    
    # 自动读取模型的标签映射 {0: 'A1', 1: 'A2'...}
    id2label = model.config.id2label
    
    model.eval()
    with torch.no_grad():
        for start in tqdm(range(0, len(texts), batch_size), desc="Evaluating"):
            batch = texts[start:start + batch_size]
            if not batch: continue

            enc = tok(
                batch,
                truncation=True,
                padding=True,
                max_length=512,
                return_tensors="pt",
            ).to(DEVICE)

            logits = model(**enc).logits
            idxs = logits.argmax(dim=-1)

            for i in range(len(batch)):
                idx = int(idxs[i].item())
                # 直接从模型配置里拿标签，不再怕顺序搞错
                label = id2label.get(idx, "Unknown")
                labels.append(label)
                
    return labels

# ========================================================
# 3. 执行清洗
# ========================================================

if __name__ == "__main__":
    tokenizer, model = load_model()

    if model:
        # 🔴 记得把这里改成你具体要清洗的文件名
        input_file = "cambridge_simplified_only.csv"
        
        if os.path.exists(input_file):
            df = pd.read_csv(input_file)
            print(f"原始数据: {len(df)} 行")
            
            # 收集任务
            tasks = []
            target_cols = [f"text_{lvl}" for lvl in ["A1", "A2", "B1", "B2", "C1", "C2"]]
            
            for idx, row in df.iterrows():
                for col in target_cols:
                    if col in df.columns and pd.notna(row[col]):
                        tasks.append((idx, col, str(row[col])))

            if tasks:
                print(f"开始验证 {len(tasks)} 个片段...")
                all_texts = [t[2] for t in tasks]
                preds = predict_batch(all_texts, tokenizer, model, batch_size=16)

                kept, dropped = 0, 0
                for i, (row_idx, col, text) in enumerate(tasks):
                    target_lvl = col.split("_")[1] # text_A1 -> A1
                    pred_lvl = preds[i]
                    
                    # 简单清洗一下标签 (有些模型输出带有空格)
                    if pred_lvl.strip() == target_lvl:
                        kept += 1
                    else:
                        df.at[row_idx, col] = None
                        dropped += 1
                
                # 保存
                df.dropna(subset=target_cols, how='all').to_csv("cambridge_verified_final.csv", index=False)
                print(f"\n🎉 成功！保留: {kept} | 删除: {dropped}")
                print("结果已保存至: cambridge_verified_final.csv")
            else:
                print("没有找到需要验证的数据列！")
        else:
            print(f"找不到文件: {input_file}")

🛡️ Running on device: cpu (Stable Mode)
Loading model: AbdulSami/bert-base-cased-cefr...
